In [1]:
# ---------------------------------------------------------------------------------------
# Main Script 
# ---------------------------------------------------------------------------------------

# ===========================
# Import Required Libraries
# ===========================
import re
import pandas as pd
import numpy as np
import pandapower.networks
import pandapower as pp
import os
import copy
import unicodedata

# ===============================
# Import Custom Function Modules
# ===============================
# load_gen_assignment.py includes:
#   - aggregated_measurements_substation()
#   - assign_load_values_from_measurements()
#   - assign_generators_values_from_measurements()
#
# rsa_engine.py includes:
#   - real_time_security_assessment()

import load_gen_assignment as lg
import rsa_engine as rs

In [ ]:
# =======================================================================
# NOTE:
# If this script was originally written in Jupyter Notebook and you want 
# to convert it to a .py script, you can use the following command in terminal:
# jupyter nbconvert --to script your_notebook.ipynb
# =======================================================================

# =================================
# Define Substation Name Mapping
# =================================
# This list is used for substation name matching between load/gen data and measurements.
substations = [
    'Åkirkeby', 'Allinge', 'Bodilsker', 'Gudhjem', 'Hasle', 'Nexø', 'Olsker',
    'Østerlars', 'Poulsker', 'Rønne Nord', 'Rønne Syd', 'Snorrebakken',
    'Svaneke', 'Værket', 'Vesthavnen', 'Viadukten', 'Dalslunde'
]

# =======================================
# Define Root Directory for File Access
# =======================================
root_dir = os.getcwd()
print("Root Directory: ", root_dir)

# ==========================================
# Load the Network Topology from Excel File
# ==========================================
# The Excel file contains the buses, lines, loads, etc. in the pandapower format.
net = pp.from_excel(root_dir + ".xlsx")

Root Directory:  c:\Users\lukas\OneDrive - Danmarks Tekniske Universitet\SYNERGIES Project -- DTU Team - Lukas Tej Marner_SA Shared Folder\DT_1_Engines\E1


pandapowerNet-version is newer than your pandapower version. Please update pandapower `pip install --upgrade pandapower`.


In [3]:
# ==================================================================
# 🔄 Step 1: Aggregate Measurement Data for the Target Time Snapshot
# ==================================================================
# This retrieves substation-level active power production and consumption.
measurement_prod_cons = lg.aggregated_measurements_substation('2022-01-01 00:15:00')

# ======================================================================
# ⚡ Step 2: Assign Active & Reactive Power Values to Loads in the Network
# ======================================================================
# Assigns p_mw and q_mvar to net.load based on measurements and a lagging power factor.
net = lg.assign_load_values_from_measurements(net, measurement_prod_cons, substations)

# ======================================================================
# ⚙️ Step 3: Assign Active & Reactive Power to Static Generators (sgen)
# ======================================================================
# Creates net.sgen and populates it based on measured substation-level production.
net = lg.assign_generators_values_from_measurements(net, measurement_prod_cons, substations)

# ==========================================================
# 🔎 Step 4: Run Real-Time Security Assessment on the Network
# ==========================================================
# This runs a base power flow, checks voltage limits, line loading, and prints results.
net = rs.real_time_security_assessment(net)


==================== Real-Time Security Assessment Engine (RSAE) ====================

[0] Mapping bus names to line endpoints...
[1] Running base case power flow...
    ✅ Power flow successful.

[2] Checking for voltage violations...
    ⚡ Voltage Violations: 20
[3] Checking for thermal overloads on lines...
    🌡️ Line Overloads: 0

📊 ================= Security Assessment Dashboard ================ 📊

🔺 Voltage Violations Detected:
    - Bus 36 (Viadukten 10kV): Voltage = 1.161 p.u.
    - Bus 37 (Gl Dampværk B 10.5 kV): Voltage = 1.201 p.u.
    - Bus 38 (Gl Dampværk C Trf 2): Voltage = 1.201 p.u.
    - Bus 39 (Værket 10kV): Voltage = 1.201 p.u.
    - Bus 40 (Gl Dampværk A 10.5 kV): Voltage = 1.201 p.u.
    - Bus 42 (Rønne Nord 10 kV): Voltage = 1.161 p.u.
    - Bus 43 (Nexø 10.5 kV): Voltage = 1.160 p.u.
    - Bus 44 (Bodilsker 10.5 kV): Voltage = 1.142 p.u.
    - Bus 45 (Poulsker 10.5 kV): Voltage = 1.159 p.u.
    - Bus 46 (Åkirkeby 10.5 kV): Voltage = 1.189 p.u.
    - Bus 47 (Snor